In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
home_dir = "../"

import pandas as pd
from utils.pandas_extented_filters import filter_remove_null_nan_empty_entries

In [3]:
gene2refseq_filtered_filepath = home_dir+"data/gene/gene2refseq_filtered.gz"
gene2refseq_filtered_df = pd.read_csv(gene2refseq_filtered_filepath, compression='gzip', comment='#', delim_whitespace=False, sep="\t")#, header=None, names=col_names)
print(gene2refseq_filtered_df.shape)
gene2refseq_filtered_df.head()

(63738, 16)


,tax_id,GeneID,status,RNA_nucleotide_accession.version,RNA_nucleotide_gi,protein_accession.version,protein_gi,genomic_nucleotide_accession.version,genomic_nucleotide_gi,start_position_on_the_genomic_accession,end_position_on_the_genomic_accession,orientation,assembly,mature_peptide_accession.version,mature_peptide_gi,Symbol
0,9606,1,REVIEWED,NM_130786.4,1653960645,NP_570602.2,21071030,NC_000019.10,568815579,58345182,58353491,-,Reference GRCh38.p14 Primary Assembly,-,-,A1BG
1,9606,2,REVIEWED,NM_000014.6,1779421316,NP_000005.3,1779421317,NC_000012.12,568815586,9067707,9116228,-,Reference GRCh38.p14 Primary Assembly,-,-,A2M
2,9606,2,REVIEWED,NM_001347423.2,1677538580,NP_001334352.2,1677538581,NC_000012.12,568815586,9067707,9116228,-,Reference GRCh38.p14 Primary Assembly,-,-,A2M
3,9606,2,REVIEWED,NM_001347424.2,1779421277,NP_001334353.2,1779421278,NC_000012.12,568815586,9067707,9116228,-,Reference GRCh38.p14 Primary Assembly,-,-,A2M
4,9606,2,REVIEWED,NM_001347425.2,1779421327,NP_001334354.2,1779421328,NC_000012.12,568815586,9067707,9116228,-,Reference GRCh38.p14 Primary Assembly,-,-,A2M


In [8]:
# clinvar_data_filepath = home_dir+"data/clinvar/search_results/clinvar_HumanPathogenicMissenseVariants01012022To14022023.txt"
clinvar_data_filepath = home_dir+"data/clinvar/search_results/clinvar_HumanLikelyPathogenicMissenseVariants01012022To14022023.txt"
clinvar_hum_patho_missense_variants_df = pd.read_csv(clinvar_data_filepath, delim_whitespace=False, sep="\t")
print(clinvar_hum_patho_missense_variants_df.shape)
print(clinvar_hum_patho_missense_variants_df.columns)
# clinvar_hum_patho_missense_variants_df.head()

(6828, 16)
Index(['Name', 'Gene(s)', 'Protein change', 'Condition(s)',
       'Clinical significance (Last reviewed)', 'Review status', 'Accession',
       'GRCh37Chromosome', 'GRCh37Location', 'GRCh38Chromosome',
       'GRCh38Location', 'VariationID', 'AlleleID(s)', 'dbSNP ID',
       'Canonical SPDI', 'Unnamed: 15'],
      dtype='object')


In [9]:
# removing variants that do not have any protein variants associated with it
clinvar_hum_patho_missense_variants_df = filter_remove_null_nan_empty_entries(clinvar_hum_patho_missense_variants_df, a_col_name="Protein change")
clinvar_hum_patho_missense_variants_df.shape

Number of NAN rows removed: 15
Number of NULL rows removed: 0
Number of empty rows removed: 0


(6813, 16)

In [10]:
clinvar_hum_patho_missense_variants_df["RNA"] = clinvar_hum_patho_missense_variants_df["Name"].apply(lambda x: x.split("(")[0]) # RNA transcript accession: NM_accession.version
merged_df = clinvar_hum_patho_missense_variants_df.merge(gene2refseq_filtered_df, how="left", left_on="RNA", right_on="RNA_nucleotide_accession.version") # mapping clinvar mRNA transcripts to protein accession

# same gene can be present in X and Y. Removing chromosome X to Y wrongly mapped variations.
merged_df["Canonical SPDI chr"] = merged_df["Canonical SPDI"].apply(lambda x:x.split(":")[0]) 
merged_df = merged_df[merged_df["Canonical SPDI chr"] == merged_df["genomic_nucleotide_accession.version"]] # not(!) in the condition will give wrongly mapped variations
print(merged_df.shape)
print(merged_df.columns)
# merged_df.loc[0]

(6813, 34)
Index(['Name', 'Gene(s)', 'Protein change', 'Condition(s)',
       'Clinical significance (Last reviewed)', 'Review status', 'Accession',
       'GRCh37Chromosome', 'GRCh37Location', 'GRCh38Chromosome',
       'GRCh38Location', 'VariationID', 'AlleleID(s)', 'dbSNP ID',
       'Canonical SPDI', 'Unnamed: 15', 'RNA', 'tax_id', 'GeneID', 'status',
       'RNA_nucleotide_accession.version', 'RNA_nucleotide_gi',
       'protein_accession.version', 'protein_gi',
       'genomic_nucleotide_accession.version', 'genomic_nucleotide_gi',
       'start_position_on_the_genomic_accession',
       'end_position_on_the_genomic_accession', 'orientation', 'assembly',
       'mature_peptide_accession.version', 'mature_peptide_gi', 'Symbol',
       'Canonical SPDI chr'],
      dtype='object')


In [14]:
nm_ids_not_mapped = set(clinvar_hum_patho_missense_variants_df["RNA"].unique()) - set(merged_df["RNA"].unique())
print(nm_ids_not_mapped)
df_removed_NNE_protChange = filter_remove_null_nan_empty_entries(merged_df, a_col_name="Protein change")
df_removed_NNE_protAccVersion = filter_remove_null_nan_empty_entries(df_removed_NNE_protChange, a_col_name="protein_accession.version")
df_removed_NNE_protAccVersion.shape

{'NM_001288953.1', 'NM_000074.2', 'NM_004646.3'}
Number of NAN rows removed: 0
Number of NULL rows removed: 0
Number of empty rows removed: 0
Number of NAN rows removed: 0
Number of NULL rows removed: 0
Number of empty rows removed: 0


(3916, 34)

In [15]:
out_filepath = home_dir+"data/clinvar/filtered/" + clinvar_data_filepath.split("/")[-1]
df_removed_NNE_protAccVersion.to_csv(out_filepath, sep="\t", index=False, header=True)